# **실습 코드**
### 차례
1. 문서를 읽고 이해하는 산업안전 AI 에이전트 만들기🤖
- 핵심 키워드: OCR, PDF, 텍스트 문서, Vector DB, Retrieval-Augmented Generation
- 실습 목표: 문서 업로드 → 임베딩 → 검색 → 질문 응답까지 흐름 구성
- 실습 내용: AI OCR로 문서(pdf, png) 읽기, Naive RAG(벡터 검색), 하이브리드 RAG(벡터+키워드 검색), 에이전트 기반 RAG
2. 자연어로 데이터를 분석하고 설명하는 AI 에이전트 만들기 (Text-to-SQL + CSV 실습)🧠
- 핵심 키워드: Text-to-SQL, 자연어 질의, CSV 분석, LLM 기반 해석
- 실습 목표: 자연어 → CSV 분석 & SQL 변환 & 결과 해석
- 실습 내용: CSV 파일(Pandas) 전용 에이전트, SQL 전용 에이전트, CSV + SQL 통합형 AI 에이전트

### 사용 데이터셋 - 수강 기업 관련(건설업) 자료로 준비
1. 중대재해처벌법.pdf (출처: 법제처)
2. 카드 영수증 이미지 - (출처: https://www.banksalad.com/contents/%EC%98%81%EC%88%98%EC%A6%9D-%EC%83%89%EA%B9%94%EC%9D%B4-%EC%9D%B4%EC%83%81%ED%95%B4%EC%9A%94-%ED%8C%8C%EB%9E%80%EC%83%89-%EC%98%81%EC%88%98%EC%A6%9D-%EC%9D%B4-%EB%8A%98%EC%96%B4%EB%82%9C-%EC%9D%B4%EC%9C%A0-dsPrc)
3. 건설 장비 데이터셋 Folklift.csv  - 지게차 렌탈 정보 데이터 (출처: https://www.kaggle.com/datasets/andersra/sales-data-from-a-rental-company?select=Mini+dumper.csv)
4. 건설 장비 데이터셋 Minidump.csv - 미니덤프 렌탈 정보 데이터 (출처: https://www.kaggle.com/datasets/andersra/sales-data-from-a-rental-company?select=Mini+dumper.csv)

# **1. 문서를 읽고 이해하는 산업안전 AI 에이전트 만들기🤖**

## 문서 [중대재해처벌법] 설명
- 중대재해처벌법 조문 내용을 포함한 법령 텍스트 문서입니다.
- pdf 파일 형태입니다.
- 데이터는 법제처에서 가져왔습니다.

## 이미지 [카드 영수증] 설명
- 카드 영수증을 카메라로 찍은 png 파일입니다.

# 1.1 Naive RAG🚀
- 가장 기본적인 RAG 구성으로, 의미 기반 임베딩 검색만 사용하는 방식입니다.
- 프롬프트 설계, 검색 전략 튜닝 없이 기본 흐름만 체험합니다.
- PDF 읽기: Upstage Document Digitization API를 사용하여 스캔 문서를 포함한 다양한 형태의 PDF에서 OCR로 텍스트 추출 경험
- 임베딩 모델: OpenAI embedding 사용 (text-embedding-3-small)
- 벡터 저장소: FAISS (로컬 저장소 사용)
- LLM: OpenAI GPT-4 (gpt-4o)

- 흐름: 문서 청크 → 임베딩 생성 → FAISS 검색 → 검색된 문서 조각을 LLM에 입력 → 답변 생성

In [ ]:
!pip install langchain-core
!pip install langchain-openai # OpenAI 연동 (LLM, Embedding)
!pip install langchain-community # 외부 도구/라이브러리 연동
!pip install langchain-upstage # Upstage 연동 (OCR 모델 사용)
!pip install faiss-cpu # FAISS (벡터 검색용, CPU 버전)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.52.4 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.19.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.9 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from google.colab import files
import gradio as gr
import requests
import os
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# PDF 파일 업로드
uploaded = files.upload()

pdf_filenames = list(uploaded.keys())
print("업로드된 파일들:", pdf_filenames)

Saving screenshot_영수증.png to screenshot_영수증.png
Saving 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf to 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf
업로드된 파일들: ['screenshot_영수증.png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf']


In [ ]:
# OCR 전문 모델을 사용하기 위해 Upstage API 키를 입력
api_key = ""

# 문서 파일들 텍스트 추출 함수
def extract_text_with_ocr(api_key, pdf_path):
    url = "https://api.upstage.ai/v1/document-digitization"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(pdf_path, "rb")}
    data = {"model": "ocr"}

    response = requests.post(url, headers=headers, files=files, data=data)

    result = response.json()
    return result.get("text", "")  # OCR 결과 텍스트 반환

# 모든 파일에 대해 텍스트 추출
all_texts = []

for fname in pdf_filenames:
    print(f"OCR 처리 중: {fname}")
    text = extract_text_with_ocr(api_key, fname)
    print(f"{fname} 길이: {len(text)}")
    all_texts.append((fname, text))

OCR 처리 중: screenshot_영수증.png
{'apiVersion': '1.1', 'confidence': 0.9344, 'metadata': {'pages': [{'height': 760, 'page': 1, 'width': 765}]}, 'mimeType': 'multipart/form-data', 'modelVersion': 'ocr-2.2.1', 'numBilledPages': 1, 'pages': [{'confidence': 0.9344, 'height': 760, 'id': 0, 'text': 'IC신용승인 ( 고 객 용) \n단말기 1A79798148 전표번호 : 020298 \n가맹점 김태준의 탕탕집 \n주 소 서울 강남구 학동로4길 12, 1,2층(논현동) \n대표자 김태준 \n사업자 : 536-37-00183 TEL : 02-511-3235 \n6, 818 원 \n금 액 \n682 원원 \n부가세 \n7 · 500 \n합 계 \nIBK 비 씨카드 \n카드번호 : 6250-03** - **** -4903(C) 일시불 \n거래일시 · 2018/01/30 12:46:24 \n승인번호 : 72 i 10079 \n매입 : 비씨카드사 가맹 : 798606607 \n알림 : EDC매출표 \n문의 : TEL) 1544-4700 \n- - \n- - -- ma \n* 감사합니다 * \nN617021027/2. 05/20170215/0510', 'width': 765, 'words': [{'boundingBox': {'vertices': [{'x': 134, 'y': 135}, {'x': 343, 'y': 133}, {'x': 344, 'y': 158}, {'x': 134, 'y': 160}]}, 'confidence': 0.9868, 'id': 0, 'text': 'IC신용승인'}, {'boundingBox': {'vertices': [{'x': 437, 'y': 128}, {'x': 454, 'y': 128}, {'x': 455, 'y': 150}

In [ ]:
# 청크 분할 - 성능에도 영향 미치므로 적당한 값을 찾아야 함
chunk_size = 500   # 한 번에 자를 문장 길이
chunk_overlap = 100   # 청크가 겹치는 길이

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# 여러 문서에 대해 반복 분할 수행
documents = []

for fname, text in all_texts:
    split_docs = splitter.create_documents(
        [text],
        metadatas=[{"source": fname}]
    )
    documents.extend(split_docs)

print(f"총 청크 개수: {len(documents)}")

총 청크 개수: 20


In [ ]:
# documents의 구조 확인: metadata, page_content
# 각 Document는 하나의 청크(문서 조각)를 나타냄
# page_content에는 청크된 텍스트가 들어있음
# metadata에는 출처 등의 부가정보가 들어감
documents

[Document(metadata={'source': 'screenshot_영수증.png'}, page_content='IC신용승인 ( 고 객 용) \n단말기 1A79798148 전표번호 : 020298 \n가맹점 김태준의 탕탕집 \n주 소 서울 강남구 학동로4길 12, 1,2층(논현동) \n대표자 김태준 \n사업자 : 536-37-00183 TEL : 02-511-3235 \n6, 818 원 \n금 액 \n682 원원 \n부가세 \n7 · 500 \n합 계 \nIBK 비 씨카드 \n카드번호 : 6250-03** - **** -4903(C) 일시불 \n거래일시 · 2018/01/30 12:46:24 \n승인번호 : 72 i 10079 \n매입 : 비씨카드사 가맹 : 798606607 \n알림 : EDC매출표 \n문의 : TEL) 1544-4700 \n- - \n- - -- ma \n* 감사합니다 * \nN617021027/2. 05/20170215/0510'),
 Document(metadata={'source': '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf'}, page_content='중대재해 처벌 등에 관한 법률 \n중대재해 처벌 등에 관한 법률 ( 약칭: 중대재해처벌법 ) \n[시행 2022. 1. 27.] [법률 제17907호, 2021. 1. 26., 제정] \n법무부 (공공형사과) 02-2110-3539 \n환경부 (화학물질정책과) 044-201-6774, 6775 \n고용노동부 (중대산업재해감독과) 044-202-8955 \n산업통상자원부 (산업일자리혁신과) 044-203-4228 \n국토교통부 (시설안전과) 044-201-4594 \n공정거래위원회 (소비자안전교육과) 044-200-4419 \n중소벤처기업부 (인력정책과) 044-204-7442, 7446 \n제1장 총칙 \n제1조(목적) 이 법은 사업 또는 사업장, 공중이용시설 및 공중교통수단을 운영하거나 인체에 해로운 원료나 제조물을 \n취급하면서 안전 ·

In [ ]:
# 첫 번째 청크 확인
doc = documents[0]
print(f"[출처: {doc.metadata.get('source')}]\n\n{doc.page_content}")

[출처: screenshot_영수증.png]

IC신용승인 ( 고 객 용) 
단말기 1A79798148 전표번호 : 020298 
가맹점 김태준의 탕탕집 
주 소 서울 강남구 학동로4길 12, 1,2층(논현동) 
대표자 김태준 
사업자 : 536-37-00183 TEL : 02-511-3235 
6, 818 원 
금 액 
682 원원 
부가세 
7 · 500 
합 계 
IBK 비 씨카드 
카드번호 : 6250-03** - **** -4903(C) 일시불 
거래일시 · 2018/01/30 12:46:24 
승인번호 : 72 i 10079 
매입 : 비씨카드사 가맹 : 798606607 
알림 : EDC매출표 
문의 : TEL) 1544-4700 
- - 
- - -- ma 
* 감사합니다 * 
N617021027/2. 05/20170215/0510


In [ ]:
# 두 번째 청크 확인
doc = documents[1]
print(f"[출처: {doc.metadata.get('source')}]\n\n{doc.page_content}")

[출처: 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf]

중대재해 처벌 등에 관한 법률 
중대재해 처벌 등에 관한 법률 ( 약칭: 중대재해처벌법 ) 
[시행 2022. 1. 27.] [법률 제17907호, 2021. 1. 26., 제정] 
법무부 (공공형사과) 02-2110-3539 
환경부 (화학물질정책과) 044-201-6774, 6775 
고용노동부 (중대산업재해감독과) 044-202-8955 
산업통상자원부 (산업일자리혁신과) 044-203-4228 
국토교통부 (시설안전과) 044-201-4594 
공정거래위원회 (소비자안전교육과) 044-200-4419 
중소벤처기업부 (인력정책과) 044-204-7442, 7446 
제1장 총칙 
제1조(목적) 이 법은 사업 또는 사업장, 공중이용시설 및 공중교통수단을 운영하거나 인체에 해로운 원료나 제조물을 
취급하면서 안전 · 보건 조치의무를 위반하여 인명피해를 발생하게 한 사업주, 경영책임자, 공무원 및 법인의 처벌


In [ ]:
# 세 번째 청크 확인
doc = documents[2]
print(f"[출처: {doc.metadata.get('source')}]\n\n{doc.page_content}") # 같은 문서에선 앞의 청크와 겹치는 부분 존재

[출처: 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf]

취급하면서 안전 · 보건 조치의무를 위반하여 인명피해를 발생하게 한 사업주, 경영책임자, 공무원 및 법인의 처벌 
등을 규정함으로써 중대재해를 예방하고 시민과 종사자의 생명과 신체를 보호함을 목적으로 한다. 
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. 
1. "중대재해"란 "중대산업재해"와 "중대시민재해"를 말한다. 
2. "중대산업재해"란 「산업안전보건법」 제2조제1호에 따른 산업재해 중 다음 각 목의 어느 하나에 해당하는 결과를 
야기한 재해를 말한다. 
가. 사망자가 1명 이상 발생 
나. 동일한 사고로 6개월 이상 치료가 필요한 부상자가 2명 이상 발생 
다. 동일한 유해요인으로 급성중독 등 대통령령으로 정하는 직업성 질병자가 1년 이내에 3명 이상 발생 
3. "중대시민재해"란 특정 원료 또는 제조물, 공중이용시설 또는 공중교통수단의 설계, 제조, 설치, 관리상의 결함을


In [ ]:
# OpenAI API 키 설정하기
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# OpenAI의 임베딩 모델을 사용하여 문서와 질문을 벡터로 변환합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# FAISS 벡터스토어를 생성합니다.
# documents: 텍스트 청크 리스트 (이미 text splitter로 나눈 상태)
# embeddings: 각 청크를 임베딩 벡터로 변환하여 벡터 인덱스를 생성
# 이렇게 생성된 FAISS 인덱스를 통해 유사도 기반 검색이 가능하게 됩니다.
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
# LLM 모델 선택
llm = ChatOpenAI(model="gpt-4-turbo",
                 temperature=0) # rag에서는 모델 내부의 지식이 아닌, 추가 문서의 지식을 검색해야 하므로 temperature=0으로 설정해서 답변 고정

# RAG 전체 흐름을 하나로 묶기 위해서 RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
# rag 기술에 따른 답변 출력 형태가 다르므로 답변 형태를 통일하기 위해서 함수를 만들어줍니다
def extract_answer(result):
    if isinstance(result, dict):
        if "output" in result:
            return result["output"]
        elif "result" in result:
            return result["result"]
        else:
            return str(result)
    else:
        return getattr(result, "content", str(result))

In [ ]:
# 질문 1 - 영수증 관련
query = "2018년 1월 30일에 어느 곳에서 카드를 얼마나 사용했는지 알려줘"
result = qa_chain.invoke({"query": query})
print("답변:\n\n", extract_answer(result))

답변:

 2018년 1월 30일에 서울 강남구 학동로4길 12, 1,2층(논현동)에 위치한 김태준의 탕탕집에서 7,500원을 IBK 비씨카드로 결제하셨습니다.


In [ ]:
# 질문 2 - 추론이 필요한 질의
query = "상시 근로자가 4명인 사업장은 중대재해처벌법의 적용을 받지 않는 이유는 무엇인가요?"
result = qa_chain.invoke({"query": query})
print("답변:\n\n", extract_answer(result))

답변:

 상시 근로자가 5명 미만인 사업장은 중대재해처벌법의 적용을 받지 않습니다. 이는 법의 적용 범위를 설정함에 있어서 소규모 사업장의 특성과 운영 상의 어려움을 고려한 것으로 보입니다. 소규모 사업장의 경우, 인력 및 자원이 제한적이어서 대규모 사업장에 비해 안전 및 보건 관리에 필요한 시스템을 갖추고 유지하기가 더 어려울 수 있습니다. 따라서, 법적 요구사항을 충족시키는 데 필요한 부담을 줄이기 위해 이러한 사업장에 대해서는 법의 적용을 제외하고 있습니다. 이는 사업장의 규모와 운영의 실질적인 조건을 고려한 조치로 볼 수 있습니다.


In [ ]:
# 질문 3 - 여러 조항을 바탕으로 추론이 필요한 질의
query = "산업재해와 시민재해의 양벌규정은 어떤 점에서 다르게 적용되나요?"
result = qa_chain.invoke({"query": query})
print("답변:\n\n", extract_answer(result))

답변:

 산업재해와 시민재해에 대한 양벌규정은 각각의 재해 유형에 따라 다르게 적용됩니다. 산업재해는 주로 사업장 내에서 발생하는 재해로, 사업주나 경영책임자 등이 안전 및 보건 조치를 소홀히 함으로써 발생하는 경우가 많습니다. 이에 따라, 사업주나 경영책임자 등이 적절한 안전 및 보건 조치를 취하지 않아 중대산업재해가 발생한 경우, 그들에게 법적 책임을 물을 수 있습니다.

반면, 시민재해는 일반 시민이 이용하는 공중이용시설이나 공중교통수단 등에서 발생하는 재해로, 설계, 제조, 설치, 관리의 결함이 주요 원인입니다. 시민재해의 경우, 해당 시설이나 제품의 제조사, 설계자, 관리자 등이 안전규정을 준수하지 않아 재해가 발생했을 때 이들에게 법적 책임을 물을 수 있습니다.

따라서, 산업재해는 주로 사업장의 안전 관리 책임자나 사업주에게 책임이 있으며, 시민재해는 제품이나 시설의 설계, 제조, 관리를 담당하는 자에게 책임이 있습니다. 이러한 차이는 각 재해의 발생 원인과 관련된 법적 책임자가 다르기 때문에 발생합니다.


# 1.2 하이브리드 RAG🚀
- 기존 Naive RAG에서는 임베딩 기반 벡터 검색만 사용했지만, 실제 문서 검색에서는 키워드 기반 검색도 함께 활용하면 성능이 향상됩니다.
- 하이브리드 RAG는 두 가지 검색 방식을 결합하여 검색 정확도를 높이는 기법입니다.

- 임베딩: OpenAI 임베딩 사용 → 의미 기반 검색 (semantic search)
- BM25: Kiwi 형태소 분석기 + BM25 → 키워드 기반 검색 (lexical search)
- Ensemble Retriever: FAISS + BM25 가중치 조합 → 두 결과를 합산하여 보완
- 흐름: 문서 청크 분할 → [(임베딩 생성 → FAISS 저장) + (형태소 분석 → BM25 인덱스 생성)] → 사용자 질문 입력 → 벡터 검색(Faiss) → 키워드 검색(BM25) → 두 검색 결과 병합 → context 생성 → LLM 프롬프트 생성
→ LLM 호출 → 답변 생성

- BM25는 단어 기반 검색 방식이기 때문에, 한국어처럼 형태소 단위로 의미가 나뉘는 언어에서는 형태소 분석기를 이용해 "실제 의미 단위의 단어"를 추출하면 검색 성능이 크게 향상됩니다.

* 추가: Re-rank (후처리 정렬): 벡터 검색과 키워드 검색 결과를 단순 결합하는 대신, LLM 또는 전용 모델을 활용해 검색된 문서들을 질문과의 관련도 순으로 재정렬합니다. 이는 의미적 정확도가 더 높은 문서를 상위에 배치하여, LLM 응답의 품질을 향상시킵니다.

In [ ]:
!pip install langchain-core
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-upstage
!pip install faiss-cpu
!pip install kiwipiepy # 한국어 토크나이저 라이브러리
!pip install rank_bm25 # 키워드 검색을 위한 라이브러리

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.1 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.21.0-py3-none-any.whl size=35593192 sha256=8db1c66e1c49d2340d5806238d6085569f38d18862c380712382bbbb1366ebcd
  Stored in directory: /root/.cache/pip/wheels/b0/16/3d/95053ab5298f0f0f22ffea6de0200b6f24bffb73cab4c1a828
Successfully built kiwipiepy_model


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from google.colab import files
from kiwipiepy import Kiwi
from rank_bm25 import BM25Okapi
import gradio as gr
import requests
import os
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 파일 업로드
uploaded = files.upload()

pdf_filenames = list(uploaded.keys())
print("업로드된 파일들:", pdf_filenames)

Saving screenshot_영수증.png to screenshot_영수증 (1).png
Saving 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf to 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (1).pdf
업로드된 파일들: ['screenshot_영수증 (1).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (1).pdf']


In [ ]:
# OCR 전문 모델을 사용하기 위해 Upstage API 키를 입력
api_key = ""

# 텍스트 추출 함수
def extract_text_with_ocr(api_key, pdf_path):
    url = "https://api.upstage.ai/v1/document-digitization"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(pdf_path, "rb")}
    data = {"model": "ocr"}

    response = requests.post(url, headers=headers, files=files, data=data)
    result = response.json()
    return result.get("text", "")  # OCR 결과 텍스트 반환

# 모든 파일에 대해 텍스트 추출
all_texts = []

for fname in pdf_filenames:
    print(f"OCR 처리 중: {fname}")
    text = extract_text_with_ocr(api_key, fname)
    print(f"{fname} 길이: {len(text)}")
    all_texts.append((fname, text))

OCR 처리 중: screenshot_영수증 (1).png
screenshot_영수증 (1).png 길이: 396
OCR 처리 중: 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (1).pdf
중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (1).pdf 길이: 7461


In [ ]:
# 청크 분할 - 성능에도 영향 미치므로 적당한 값을 찾아야 함
chunk_size = 500   # 한 번에 자를 문장 길이
chunk_overlap = 100   # 청크가 겹치는 길이

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# 여러 문서에 대해 반복 분할 수행
documents = []

for fname, text in all_texts:
    split_docs = splitter.create_documents(
        [text],
        metadatas=[{"source": fname}]
    )
    documents.extend(split_docs)

print(f"총 청크 개수: {len(documents)}")

총 청크 개수: 20


In [ ]:
# 첫 번째 청크 확인
doc = documents[0]
print(f"[출처: {doc.metadata.get('source')}]\n\n{doc.page_content}")

[출처: screenshot_영수증 (1).png]

IC신용승인 ( 고 객 용) 
단말기 1A79798148 전표번호 : 020298 
가맹점 김태준의 탕탕집 
주 소 서울 강남구 학동로4길 12, 1,2층(논현동) 
대표자 김태준 
사업자 : 536-37-00183 TEL : 02-511-3235 
6, 818 원 
금 액 
682 원원 
부가세 
7 · 500 
합 계 
IBK 비 씨카드 
카드번호 : 6250-03** - **** -4903(C) 일시불 
거래일시 · 2018/01/30 12:46:24 
승인번호 : 72 i 10079 
매입 : 비씨카드사 가맹 : 798606607 
알림 : EDC매출표 
문의 : TEL) 1544-4700 
- - 
- - -- ma 
* 감사합니다 * 
N617021027/2. 05/20170215/0510


In [ ]:
# OPEN AI의 API KEY 입력
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
# 참고 - 한국어 토크나이저 Kiwi로 토큰화한 결과 보기
# Kiwi 객체 생성
kiwi = Kiwi()

# 분석할 문장
sentence = "오늘은 날씨가 매우 좋습니다."

# 형태소 분석
result = kiwi.analyze(sentence)

# 결과 출력
for token in result[0][0]:
    print(f"형태소: {token.form}, 품사: {token.tag}")

형태소: 오늘, 품사: NNG
형태소: 은, 품사: JX
형태소: 날씨, 품사: NNG
형태소: 가, 품사: JKS
형태소: 매우, 품사: MAG
형태소: 좋, 품사: VA
형태소: 습니다, 품사: EF
형태소: ., 품사: SF


In [ ]:
# BM25 + Kiwi 형태소 분석 기반 키워드 검색 클래스를 생성합니다
class KiwiBM25Retriever:
    def __init__(self, documents):
        self.documents = documents
        self.corpus = [doc.page_content for doc in documents]
        self.kiwi = Kiwi()
        self.tokenized_corpus = [self.tokenize(text) for text in self.corpus]
        self.bm25 = BM25Okapi(self.tokenized_corpus)  # BM25 인덱스 생성

    def tokenize(self, text):
        # 형태소 단위로 토큰화
        return [token.form for token in self.kiwi.tokenize(text)]

    def get_relevant_documents(self, query, top_k=5):

        # 검색 쿼리도 형태소 단위로 토큰화
        query_tokens = self.tokenize(query)
        scores = self.bm25.get_scores(query_tokens)

        # BM25 점수 상위 top_k개 선택
        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        return [(self.documents[i], scores[i]) for i in top_indices]

# BM25 검색기를 생성합니다
kiwi_bm25_retriever = KiwiBM25Retriever(documents)

# 임베딩 기반 의미 검색을 FAISS로 수행하고, 검색 결과를 유사도 점수 형태로 반환하는 함수를 만들어줍니다
def faiss_search(query, top_k=3):
    docs_and_scores = vectorstore.similarity_search_with_score(query, k=top_k) # 유사도 점수
    return [(doc, 1-score) for doc, score in docs_and_scores] # FAISS는 거리(distance)를 반환하므로 1 - 거리로 유사도로 변환 (단순 가중치 조합을 위해)

In [ ]:
# 하이브리드 검색: 벡터 검색(FAISS) + 키워드 검색(BM25)을 결합
# 가중치 weight는 조절 가능합니다

def hybrid_search(query, top_k=3, faiss_weight=0.7, bm25_weight=0.3):

    faiss_results = faiss_search(query, top_k) # 의미 기반
    bm25_results = kiwi_bm25_retriever.get_relevant_documents(query, top_k) # 키워드 기반

    combined_scores = {}  # 검색 결과 통합합니다

    # FAISS 결과에 가중치 적용하여 점수 기록
    for doc, score in faiss_results:
        combined_scores[doc.page_content] = faiss_weight * score

    # BM25 결과에 가중치 적용하여 점수 누적 (FAISS 결과와 중복될 수 있으므로 합산)
    for doc, score in bm25_results:
        if doc.page_content in combined_scores:
            combined_scores[doc.page_content] += bm25_weight * score
        else:
            combined_scores[doc.page_content] = bm25_weight * score

    # 통합된 점수를 기준으로 최종 정렬
    sorted_docs = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    # 높은 점수의 상위 top_k개의 문서를 반환합니다
    return [Document(page_content=doc, metadata={"source": pdf_filenames}) for doc, score in sorted_docs[:top_k]]

In [ ]:
# 답변 생성을 위한 LLM을 가져옵니다
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# 파이프라인을 생성합니다
def rag_pipeline_simple(query):

    # 하이브리드 검색으로 관련 청크 찾기
    hybrid_results = hybrid_search(query, top_k=5)

    # 검색된 청크들을 하나로 연결하면서 출처 표시
    context_list = []
    for doc in hybrid_results:
        source = doc.metadata.get("source", "출처 정보 없음")  # 파일명 등 메타데이터
        content = doc.page_content
        context_list.append(f"[출처: {source}]\n{content}")

    context = "\n\n".join(context_list)

    # LLM용 프롬프트 생성 - 참고 문서를 제시하도록 지시: 신뢰성 높일 수 있게 함
    prompt = f"""
다음 참고 정보를 기반으로 질문에 답하세요.
가능하면 참고 정보의 출처도 함께 제시하세요.
정보에 없으면 '정보 없음' 이라고 답변하세요.

참고 정보:
{context}

질문: {query}
답변:
"""

    # LLM에 프롬프트를 전달합니다
    response = llm.invoke(prompt)
    return response.content

In [ ]:
# 답변 형태를 통일하기 위한 함수
def extract_answer(result):
    if isinstance(result, dict):
        if "output" in result:
            return result["output"]
        elif "result" in result:
            return result["result"]
        else:
            return str(result)
    else:
        return getattr(result, "content", str(result))

In [ ]:
# 질문1 - 영수증 읽기
query = "2018년 1월 30일에 어느 곳에서 카드를 얼마나 사용했는지 알려줘"
result = rag_pipeline_simple(query)
print("답변:\n\n", extract_answer(result))

답변:

 2018년 1월 30일에 서울 강남구 학동로4길 12, 1,2층(논현동)에 위치한 '김태준의 탕탕집'에서 IBK 비씨카드로 총 7,500원을 사용했습니다.


In [ ]:
# 질문2 - 추론이 필요한 질의
query = "상시 근로자가 4명인 사업장은 중대재해처벌법의 적용을 받지 않는 이유는 무엇인가요?"
result = rag_pipeline_simple(query)
print("답변:\n\n", extract_answer(result))

답변:

 상시 근로자가 4명인 사업장은 중대재해처벌법의 적용을 받지 않는 이유는, 중대재해 처벌 등에 관한 법률 제3조(적용범위)에 따라 상시 근로자가 5명 미만인 사업 또는 사업장의 사업주(개인사업주에 한정한다) 또는 경영책임자등에게는 이 장의 규정을 적용하지 않기 때문입니다. 이는 해당 법률의 규정에 명시되어 있습니다.


In [ ]:
# 질문3 - 여러 조항을 바탕으로 추론이 필요한 질의
query = "산업재해와 시민재해의 양벌규정은 어떤 점에서 다르게 적용되나요?"
result = rag_pipeline_simple(query)
print("답변:\n\n", extract_answer(result))

답변:

 산업재해와 시민재해의 양벌규정은 벌금의 액수에서 차이를 보입니다.

산업재해의 경우, 중대산업재해에 대한 양벌규정에 따라 법인 또는 기관에 부과되는 벌금은 다음과 같습니다:
- 제6조제1항의 경우: 50억원 이하의 벌금
- 제6조제2항의 경우: 10억원 이하의 벌금

시민재해의 경우, 중대시민재해에 대한 양벌규정에 따라 법인 또는 기관에 부과되는 벌금은 다음과 같습니다:
- 제10조제1항의 경우: 50억원 이하의 벌금

이 정보는 "중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (1).pdf"에서 확인할 수 있습니다.


In [ ]:
# Rerank 추가한 버전과 비교해봅니다
# Rerank 사용하면 일반적으로 답변의 정확도가 향상됩니다
# 하지만 일반적으로 답변 속도는 느려집니다

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

def rerank(query, documents):
    docs_with_source = "\n\n".join(
        [f"[출처: {doc.metadata.get('source', '출처 정보 없음')}]\n{doc.page_content}" for doc in documents]
    )

    prompt = f"""
다음은 사용자의 질문과 검색된 문서 조각입니다.

질문: "{query}"

문서 조각:
{docs_with_source}

가장 관련성 높은 문서를 1개만 골라 그 문서 내용을 그대로 출력하세요. 만약 관련성이 비슷하다면 첫 번째 문서를 출력하세요.
"""
    response = llm.invoke(prompt)
    return response.content

def rag_pipeline_with_rerank(query):
    hybrid_results = hybrid_search(query, top_k=5)

    reranked_context = rerank(query, hybrid_results)

    final_prompt = f"""
다음 참고 정보를 기반으로 질문에 답하세요.
답변 작성 시 반드시 참고 정보 안에 존재하는 출처 파일명을 그대로 복사하여 출력하세요.
출처 정보를 임의로 생성하거나 변형하지 말고, 참고 정보 안에 제공된 출처 텍스트를 정확히 사용하세요.
정보에 없으면 '정보 없음' 이라고 답변하세요.

참고 정보:
{reranked_context}

질문: {query}
답변:
"""

    response = llm.invoke(final_prompt)
    return response.content

In [ ]:
# 질문2
query = "상시 근로자가 4명인 사업장은 중대재해처벌법의 적용을 받지 않는 이유는 무엇인가요?"
result = rag_pipeline_with_rerank(query)
print("답변:\n\n", extract_answer(result))

답변:

 제3조(적용범위)에 따르면 상시 근로자가 5명 미만인 사업 또는 사업장의 사업주(개인사업주에 한정한다. 이하 같다) 또는 경영책임자등에게는 이 장의 규정을 적용하지 아니한다고 명시되어 있습니다. 따라서 상시 근로자가 4명인 사업장은 중대재해처벌법의 적용을 받지 않습니다.


⚠️출처 표시를 포함하도록 프롬프트를 설계했으나, rerank에서는 일부 질문에서는 언어 모델이 출처 파일명을 복사하지 않고 [출처: 파일명] 형태로 일반화해 출력하거나 출처를 밝히지 않는 경우가 발생합니다. 이러한 경우에는 후처리로 선택된 문서의 출처를 별도로 매핑하여 보완할 수 있습니다.⚠️

In [ ]:
# 질문3
query = "산업재해와 시민재해의 양벌규정은 어떤 점에서 다르게 적용되나요?"
result = rag_pipeline_with_rerank(query)
print("답변:\n\n", extract_answer(result))

답변:

 정보 없음


⚠️Rerank를 추가하면 일반적으로 답변의 정확도가 높아집니다. 다만, 질문과 덜 관련된 문서가 rerank 과정에서 상위로 정렬되면,
LLM이 실제로 필요한 정보를 받지 못해 답변을 제대로 생성하지 못하는 경우가 생길 수 있습니다.⚠️

# 1.3 에이전트 기반 RAG🚀
- Agent: LangChain Agent (ReAct 기반)

- Retriever Tool: 2단계에서 만든 하이브리드 검색기 등록

- 흐름: Agent가 retriever를 활용해서 답변 생성

- 특징: 신뢰성있는 검색을 위해 출처 명시

In [ ]:
!pip install langchain-core
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-upstage
!pip install faiss-cpu
!pip install kiwipiepy
!pip install rank_bm25

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.tools import Tool
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor
from google.colab import files
from kiwipiepy import Kiwi
from rank_bm25 import BM25Okapi
import gradio as gr
import requests
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 파일 업로드
uploaded = files.upload()

pdf_filenames = list(uploaded.keys())
print("업로드된 파일들:", pdf_filenames)

Saving screenshot_영수증.png to screenshot_영수증 (2).png
Saving 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf to 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf
업로드된 파일들: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']


In [ ]:
# OCR 전문 모델을 사용하기 위해 Upstage API 키를 입력
api_key = ""

# 텍스트 추출 함수
def extract_text_with_ocr(api_key, pdf_path):
    url = "https://api.upstage.ai/v1/document-digitization"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(pdf_path, "rb")}
    data = {"model": "ocr"}

    response = requests.post(url, headers=headers, files=files, data=data)
    result = response.json()
    return result.get("text", "")  # OCR 결과 텍스트 반환

# 모든 파일에 대해 텍스트 추출
all_texts = []

for fname in pdf_filenames:
    print(f"OCR 처리 중: {fname}")
    text = extract_text_with_ocr(api_key, fname)
    print(f"{fname} 길이: {len(text)}")
    all_texts.append((fname, text))

OCR 처리 중: screenshot_영수증 (2).png
screenshot_영수증 (2).png 길이: 396
OCR 처리 중: 중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf
중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf 길이: 7461


In [ ]:
# 청크 분할 - 성능에도 영향 미치므로 적당한 값을 찾아야 함
chunk_size = 500   # 한 번에 자를 문장 길이
chunk_overlap = 100   # 청크가 겹치는 길이

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# 여러 문서에 대해 반복 분할 수행
documents = []

for fname, text in all_texts:
    split_docs = splitter.create_documents(
        [text],
        metadatas=[{"source": fname}]
    )
    documents.extend(split_docs)

print(f"총 청크 개수: {len(documents)}")

총 청크 개수: 20


In [ ]:
# 첫 번째 청크 확인
doc = documents[0]
print(f"[출처: {doc.metadata.get('source')}]\n\n{doc.page_content}")

[출처: screenshot_영수증 (2).png]

IC신용승인 ( 고 객 용) 
단말기 1A79798148 전표번호 : 020298 
가맹점 김태준의 탕탕집 
주 소 서울 강남구 학동로4길 12, 1,2층(논현동) 
대표자 김태준 
사업자 : 536-37-00183 TEL : 02-511-3235 
6, 818 원 
금 액 
682 원원 
부가세 
7 · 500 
합 계 
IBK 비 씨카드 
카드번호 : 6250-03** - **** -4903(C) 일시불 
거래일시 · 2018/01/30 12:46:24 
승인번호 : 72 i 10079 
매입 : 비씨카드사 가맹 : 798606607 
알림 : EDC매출표 
문의 : TEL) 1544-4700 
- - 
- - -- ma 
* 감사합니다 * 
N617021027/2. 05/20170215/0510


In [ ]:
# API 키 입력
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 임베딩 모델 선택
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터스토어 생성
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
# LLM 모델 선택
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

In [ ]:
# BM25 + Kiwi 형태소 분석 기반 키워드 검색 클래스
class KiwiBM25Retriever:
    def __init__(self, documents):
        self.documents = documents
        self.corpus = [doc.page_content for doc in documents]
        self.kiwi = Kiwi()
        self.tokenized_corpus = [self.tokenize(text) for text in self.corpus]
        self.bm25 = BM25Okapi(self.tokenized_corpus)

    def tokenize(self, text):

        return [token.form for token in self.kiwi.tokenize(text)]

    def get_relevant_documents(self, query, top_k=5):

        query_tokens = self.tokenize(query)
        scores = self.bm25.get_scores(query_tokens)

        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        return [(self.documents[i], scores[i]) for i in top_indices]

# BM25 검색기 생성
kiwi_bm25_retriever = KiwiBM25Retriever(documents)

# FAISS 벡터 검색 (임베딩 기반 의미 검색)
def faiss_search(query, top_k=3):

    docs_and_scores = vectorstore.similarity_search_with_score(query, k=top_k)

    return [(doc, 1-score) for doc, score in docs_and_scores]

In [ ]:
# 하이브리드 검색: 벡터 검색(FAISS) + 키워드 검색(BM25)을 결합
def hybrid_search(query, top_k=3, faiss_weight=0.7, bm25_weight=0.3):

    faiss_results = faiss_search(query, top_k)
    bm25_results = kiwi_bm25_retriever.get_relevant_documents(query, top_k)

    combined_scores = {}  # 검색 결과 통합

    # FAISS 결과에 가중치 적용하여 점수 기록
    for doc, score in faiss_results:
        combined_scores[doc.page_content] = faiss_weight * score

    # BM25 결과에 가중치 적용하여 점수 누적 (FAISS 결과와 중복될 수 있으므로 합산)
    for doc, score in bm25_results:
        if doc.page_content in combined_scores:
            combined_scores[doc.page_content] += bm25_weight * score
        else:
            combined_scores[doc.page_content] = bm25_weight * score

    # 통합된 점수를 기준으로 최종 정렬 (내림차순: 높은 점수 우선)
    sorted_docs = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    # 상위 top_k개의 문서를 반환
    return [Document(page_content=doc, metadata={"source": pdf_filenames}) for doc, score in sorted_docs[:top_k]]

In [ ]:
# hybrid_search → retriever tool로 wrapping합니다
def hybrid_search_tool(query: str) -> str:
    docs = hybrid_search(query, top_k=3)
    context_list = []
    for doc in docs:
        source = doc.metadata.get("source", "출처 정보 없음")
        content = doc.page_content
        context_list.append(f"[출처: {source}]\n{content}")
    context = "\n\n".join(context_list)
    return context

# Tool 등록
retrieval_tool = Tool.from_function(
    name="HybridDocumentSearch",
    description="문서 검색을 수행하는 도구입니다. 사용자의 질문과 관련된 내용을 검색합니다.",
    func=hybrid_search_tool
)

In [ ]:
# 시스템 프롬프트에 추론을 지시합니다
# 출처 명시를 명확하게 요구합니다
prompt = ChatPromptTemplate.from_template("""
사용자의 질문에 대해 검색 도구를 활용해 최대한 정확히 답변하세요.
답변 작성 시 반드시 참고 정보 안에 존재하는 출처 파일명을 그대로 복사하여 출력하세요.
출처 정보를 임의로 생성하거나 변형하지 말고, 참고 정보 안에 제공된 출처 텍스트를 정확히 사용하세요.
정보에 없으면 '정보 없음' 이라고 답변하세요.

질문: {input}

{agent_scratchpad}
""")

# ReAct Agent 생성
agent = create_openai_functions_agent(llm, [retrieval_tool], prompt)

In [ ]:
# Agent는 Executor를 통해 실행해야 동작합니다
agent_executor = AgentExecutor(agent=agent, tools=[retrieval_tool], verbose=True)

In [ ]:
# 답변 형태를 통일하기 위한 함수
def extract_answer(result):
    if isinstance(result, dict):
        if "output" in result:
            return result["output"]
        elif "result" in result:
            return result["result"]
        else:
            return str(result)
    else:
        return getattr(result, "content", str(result))

In [ ]:
# 질문1 - 영수증 관련
query = "2018년 1월 30일에 어느 곳에서 카드를 사용했는지 알려줘"
result = agent_executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `HybridDocumentSearch` with `2018년 1월 30일 카드 사용`


[출처: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']]
IC신용승인 ( 고 객 용) 
단말기 1A79798148 전표번호 : 020298 
가맹점 김태준의 탕탕집 
주 소 서울 강남구 학동로4길 12, 1,2층(논현동) 
대표자 김태준 
사업자 : 536-37-00183 TEL : 02-511-3235 
6, 818 원 
금 액 
682 원원 
부가세 
7 · 500 
합 계 
IBK 비 씨카드 
카드번호 : 6250-03** - **** -4903(C) 일시불 
거래일시 · 2018/01/30 12:46:24 
승인번호 : 72 i 10079 
매입 : 비씨카드사 가맹 : 798606607 
알림 : EDC매출표 
문의 : TEL) 1544-4700 
- - 
- - -- ma 
* 감사합니다 * 
N617021027/2. 05/20170215/0510

[출처: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']]
취급하면서 안전 · 보건 조치의무를 위반하여 인명피해를 발생하게 한 사업주, 경영책임자, 공무원 및 법인의 처벌 
등을 규정함으로써 중대재해를 예방하고 시민과 종사자의 생명과 신체를 보호함을 목적으로 한다. 
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. 
1. "중대재해"란 "중대산업재해"와 "중대시민재해"를 말한다. 
2. "중대산업재해"란 「산업안전보건법」 제2조제1호에 따른 산업재해 중 다음 각 목의 어느 하나에 해당하는 결과를 
야기한 재해를 말한다. 
가. 사망자가 1명 이상 발생 
나. 동일한 사고로 6개월 이상 치료가 필요한 부상자가 2명 이상 발생 
다. 동

In [ ]:
# 질문2 - 추론이 필요한 질의
query = "상시 근로자가 4명인 사업장은 중대재해처벌법의 적용을 받지 않는 이유는 무엇인가요?"
result = agent_executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `HybridDocumentSearch` with `상시 근로자가 4명인 사업장 중대재해처벌법 적용`


[출처: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']]
법률」 제4조부터 제6조까지의 규정에 따라 지정된 공공기관의 장 
제2장 중대산업재해 
제3조(적용범위) 상시 근로자가 5명 미만인 사업 또는 사업장의 사업주(개인사업주에 한정한다. 이하 같다) 또는 경영책 
임자등에게는 이 장의 규정을 적용하지 아니한다. 
제4조(사업주와 경영책임자등의 안전 및 보건 확보의무) ① 사업주 또는 경영책임자등은 사업주나 법인 또는 기관이 실 
질적으로 지배 · 운영 · 관리하는 사업 또는 사업장에서 종사자의 안전 · 보건상 유해 또는 위험을 방지하기 위하여 
그 사업 또는 사업장의 특성 및 규모 등을 고려하여 다음 각 호에 따른 조치를 하여야 한다. 
1. 재해예방에 필요한 인력 및 예산 등 안전보건관리체계의 구축 및 그 이행에 관한 조치 
2. 재해 발생 시 재발방지 대책의 수립 및 그 이행에 관한 조치 
3. 중앙행정기관 · 지방자치단체가 관계 법령에 따라 개선, 시정 등을 명한 사항의 이행에 관한 조치

[출처: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']]
별로 국회 소관 상임위원회에 보고하여야 한다. 
[시행일 : 2021. 1. 26.] 제16조 
부칙 <제17907호,2021. 1. 26.> 
제1조(시행일) ① 이 법은 공포 후 1년이 경과한 날부터 시행한다. 다만, 이 법 시행 당시 개인사업자 또는 상시 근로자 
가 50명 미만인 사업 또는 사업장(건설업의 경우에는 공사금액 50억원 미만의 공사)에 대해서는 공포 후 3년이 경 
과한 날부터 시행한다. 
법제처 4 국가법령정보센터

In [ ]:
# 질문3
query = "산업재해와 시민재해의 양벌규정은 어떤 점에서 다르게 적용되나요?"
result = agent_executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `HybridDocumentSearch` with `산업재해와 시민재해의 양벌규정 차이`


[출처: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']]
제10조(중대시민재해 사업주와 경영책임자등의 처벌) ① 제9조를 위반하여 제2조제3호가목의 중대시민재해에 이르게 
한 사업주 또는 경영책임자등은 1년 이상의 징역 또는 10억원 이하의 벌금에 처한다. 이 경우 징역과 벌금을 병과할 
수 있다. 
② 제9조를 위반하여 제2조제3호나목 또는 다목의 중대시민재해에 이르게 한 사업주 또는 경영책임자등은 7년 이 
하의 징역 또는 1억원 이하의 벌금에 처한다. 
제11조(중대시민재해의 양벌규정) 법인 또는 기관의 경영책임자등이 그 법인 또는 기관의 업무에 관하여 제10조에 해 
당하는 위반행위를 하면 그 행위자를 벌하는 외에 그 법인 또는 기관에게 다음 각 호의 구분에 따른 벌금형을 과(科 
)한다. 다만, 법인 또는 기관이 그 위반행위를 방지하기 위하여 해당 업무에 관하여 상당한 주의와 감독을 게을리하 
지 아니한 경우에는 그러하지 아니하다. 
1. 제10조제1항의 경우: 50억원 이하의 벌금

[출처: ['screenshot_영수증 (2).png', '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127) (2).pdf']]
리하는 책임이 있는 경우에 한정한다. 
제6조(중대산업재해 사업주와 경영책임자등의 처벌) ① 제4조 또는 제5조를 위반하여 제2조제2호가목의 중대산업재해 
에 이르게 한 사업주 또는 경영책임자등은 1년 이상의 징역 또는 10억원 이하의 벌금에 처한다. 이 경우 징역과 벌금 
을 병과할 수 있다. 
② 제4조 또는 제5조를 위반하여 제2조제2호나목 또는 다목의 중대산업재해에 이르게 한 사업주 또는 경영책임자 
등은 7년 이하의 징역 또는 1억원 이하의 

In [ ]:
# Gradio를 사용해서 UI를 만들어 채팅을 할 수 있게 만들어봅니다

# # RAG 파이프라인 함수 선택 # naive rag의 경우 사용 코드
# def ask_rag(query):
#     output = qa_chain(query)
#     return output['result']

# # RAG 파이프라인 함수 선택 # 하이브리드의 경우 사용 코드
# def ask_rag(query):
#     return rag_pipeline_simple(query)

# # RAG 파이프라인 함수 선택 # rerank의 경우 사용 코드
# def ask_rag(query):
#     return rag_pipeline_with_rerank(query)

# RAG 파이프라인 함수 선택 # 에이전트의 경우 사용 코드
def ask_rag(query):
    result = agent_executor.invoke({"input": query})
    return result.get("output", "⚠️ 답변 생성 중 오류 발생")

# 사용자 정의 CSS
custom_css = """
#submit-btn {
    background-color: black;
    color: white;
    font-weight: bold;
    transition: background-color 0.3s ease;
}
#submit-btn:hover {
    background-color: #ff69b4;
    color: white;
}
.title-text {
    text-align: center;
    font-size: 16px;
    font-weight: bold;
    margin-top: 10px;
}
.subtitle-text {
    text-align: center;
    font-size: 16px;
    margin-bottom: 20px;
}
label {
    font-weight: bold !important;
}
"""

# Gradio UI 구성
with gr.Blocks(css=custom_css) as demo:

    # 상단 제목 및 설명
    gr.Markdown("🤖산업안전 AI 에이전트🤖", elem_classes="title-text")
    gr.Markdown("문서 내용을 기반으로 질문에 답변합니다.", elem_classes="subtitle-text")

    # 질문 입력창과 버튼
    with gr.Row():
        input_text = gr.Textbox(label="📝질문을 입력하세요", scale=4)
        submit_btn = gr.Button("답변 생성", elem_id="submit-btn", scale=1)

    # 결과 출력
    output_text = gr.Textbox(label="✨AI 답변", lines=10)

    # 버튼 클릭 시 RAG 함수 호출
    submit_btn.click(fn=ask_rag, inputs=input_text, outputs=output_text)

# 앱 실행
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ab05ce0f140775a9b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **2. 자연어로 데이터를 분석하고 설명하는 AI 에이전트 만들기 (CSV + Text-to-SQL실습)🧠**

## 데이터셋 [Forklift]와 [Minidump] 설명

- 이 두 개의 데이터셋은 건설 현장 장비 렌탈 기록에 대한 정보입니다. 지게차(forklift)와 미니덤프(minidump) 장비가 언제 주문되었고, 언제부터 언제까지 렌탈되었으며, 어떤 공급업체(Vendor)를 통해 어떤 고객(Customer)에게 대여되었는지와 같은 핵심 정보를 제공합니다.
- 데이터셋은 캐글에서 가져왔습니다. (https://www.kaggle.com/datasets/andersra/sales-data-from-a-rental-company?select=Mini+dumper.csv)

주요 컬럼 정보
- 제품ID (Product ID): 렌탈된 장비의 고유 식별 번호.
- 주문생성일 (Order created): 고객이 렌탈 주문을 접수한 날짜.
- 렌탈시작일 (Rental start): 장비 렌탈이 실제로 시작된 날짜.
- 렌탈종료일 (Rental end): 장비 렌탈이 종료된 날짜.
- 판매사ID (Vendor ID): 장비를 제공한 공급업체의 고유 식별 번호.
- 고객ID (Customer ID): 장비를 렌탈한 고객의 고유 식별 번호.

이번 실습에선 CSV파일을 기반으로 데이터 분석을 하는 경우에는 지게차 파일을, SQL파일을 기반으로 데이터 분석을 하는 경우에는 미니덤프 파일을 사용할 것입니다.

# 2.1 CSV 파일(Pandas) 분석하기🚀🚀

### Pandas 에이전트는 데이터가 이미 메모리에 로드되어 있음

--> 바로 데이터 조작과 계산 수행

--> 중간 검증 단계가 적음

--> 결과가 더 직접적으로 나옴

In [ ]:
!pip install langchain # RAG 프레임워크
!pip install langchain-openai # 랭체인이 지원하는 언어모델 중에서 OPEN AI모델 선택
!pip install langchain-community
!pip install langchain-experimental
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from google.colab import files
import warnings
import logging
import re
import os

warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# 로컬에 있는 CSV 파일을 코랩에 업로드하는 방법
# 업로드할 파일 선택
uploaded = files.upload()

# 지게차 파일 업로드
forklift_filename = list(uploaded.keys())[0]

# 업로드 확인하기
print("---> 지게차 파일 성공적으로 업로드!:", forklift_filename)

Saving Forklift.csv to Forklift.csv
---> 지게차 파일 성공적으로 업로드!: Forklift.csv


In [ ]:
# CSV 파일을 데이터프레임으로 불러오기
forklift_df = pd.read_csv(forklift_filename)

# 데이터 상위 5개 행 미리 보기
forklift_df.head()

,Product ID,Order created,Rental start,Rental end,Vendor ID,Customer ID
0,100001139,2009-11-09 10:34:22.5370000,2009-11-08 23:00:00.0000000,2010-05-09 21:59:59.0000000,144,184514
1,100001139,2009-11-09 10:42:02.7800000,2009-11-08 23:00:00.0000000,2010-05-09 21:59:59.0000000,144,184514
2,100001139,2010-01-12 09:35:57.3200000,2010-01-11 23:00:00.0000000,2010-04-09 21:59:59.0000000,144,184514
3,100001139,2011-02-23 10:39:24.4070000,2011-08-07 22:00:00.0000000,2012-03-13 22:59:59.0000000,144,184514
4,100001139,2009-08-26 09:04:53.2000000,2009-08-25 22:00:00.0000000,2009-08-26 21:59:59.0000000,144,184513


In [ ]:
# 데이터 전처리 - 컬럼명을 한국어로 바꾸기
forklift_df = forklift_df.rename(columns={
    'Product ID': '제품ID',
    'Order created': '주문생성일',
    'Rental start': '렌탈시작일',
    'Rental end': '렌탈종료일',
    'Vendor ID': '판매사ID',
    'Customer ID': '고객ID'
})

In [ ]:
# 컬럼 정보 확인
forklift_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4731 entries, 0 to 4730
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품ID    4731 non-null   int64 
 1   주문생성일   4731 non-null   object
 2   렌탈시작일   4731 non-null   object
 3   렌탈종료일   4731 non-null   object
 4   판매사ID   4731 non-null   int64 
 5   고객ID    4731 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 221.9+ KB


In [ ]:
# 데이터셋 전처리 - 데이터타입 변경 필요함
forklift_df['제품ID'] = forklift_df['제품ID'].astype('string')  # 문자 정보로 변경
forklift_df['주문생성일'] = pd.to_datetime(forklift_df['주문생성일']) # 날짜 정보로 변경
forklift_df['렌탈시작일'] = pd.to_datetime(forklift_df['렌탈시작일'])
forklift_df['렌탈종료일'] = pd.to_datetime(forklift_df['렌탈종료일'])
forklift_df['판매사ID'] = forklift_df['판매사ID'].astype('string')
forklift_df['고객ID'] = forklift_df['고객ID'].astype('string')

In [ ]:
# OpenAI API 키 설정하기
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 재설치 # 이 부분을 하지 않으면 오류가 나올 수도 있습니다
!pip install --upgrade langchain langchain-community langchain-openai langchain-experimental --quiet

In [ ]:
# OpenAI의 LLM 연결
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# 사용자 맞춤형 프롬프트 설정 가능
# 검색 결과의 품질을 높이기 위해 근거를 제시하도록 합니다
# 자율적인 문제 해결을 위해 에러 처리 능력 부여합니다
pandas_custom_prefix = """
당신은 전문적인 데이터 분석가입니다. 다음 규칙을 반드시 따르세요:
1. 모든 답변은 한국어로 작성하세요.
2. 코드를 실행하기 전에 먼저 데이터를 확인하세요.
3. 단계별로 분석을 진행하고 각 단계를 설명하세요.
4. 최종 출력은 반드시 아래 형식으로 작성하세요:

**분석 결과: (최종 결론 한 문장)

**분석 근거: (결론에 도달한 이유 설명)

**실행된 파이썬 코드: (Action Input에 작성한 코드를 그대로 작성)

5. 오류가 발생하면 다른 방법을 시도하세요.
"""

In [ ]:
# DataFrame을 분석할 수 있는 에이전트 생성
pandas_agent = create_pandas_dataframe_agent(
    llm,
    forklift_df,
    verbose=True,
    allow_dangerous_code=True,
    prefix=pandas_custom_prefix, # 프롬프트에 prefix를 직접 전달
    return_intermediate_steps=True, # 중간 실행 정보 포함(LLM이 작성한 코드를 제시하기 위해서)
    handle_parsing_errors=True
)

In [ ]:
# 질문1: 단순 집계 - 총 판매자의 수 구하기
question = "총 판매자의 수를 알려줘"
result = pandas_agent.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...
Thought: 데이터에서 '판매사ID' 열을 사용하여 고유한 판매자의 수를 계산할 수 있습니다. 이를 위해 pandas의 unique() 함수와 len() 함수를 사용할 것입니다.

Action: python_repl_ast
Action Input: len(df['판매사ID'].unique())50I now know the final answer.

Final Answer:
**분석 결과:** 총 50명의 판매자가 있습니다.

**분석 근거:** '판매사ID' 열에서 고유한 ID의 수를 계산하여 판매자의 수를 확인했습니다.

**실행된 파이썬 코드:** len(df['판매사ID'].unique())

> Finished chain.

최종 답변: **분석 결과:** 총 50명의 판매자가 있습니다.

**분석 근거:** '판매사ID' 열에서 고유한 ID의 수를 계산하여 판매자의 수를 확인했습니다.

**실행된 파이썬 코드:** len(df['판매사ID'].unique())


In [ ]:
# 파이썬 코드로 정답 직접 확인해보기
# df는 forklift_df로 변환하여 확인합니다
print("\n--- 직접 실행할 파이썬 코드 ---\n")
len(forklift_df['판매사ID'].unique())


--- 직접 실행할 파이썬 코드 ---



50

In [ ]:
# 질문 2 - 조건 연산
question = "2023년 이후 주문된 렌탈 건수는 몇 건인가요?"
result = pandas_agent.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...
Thought: 데이터를 확인한 결과, 주문생성일이 '2009-11-09 10:34:22.537000'와 같은 형식으로 제공되고 있습니다. 이를 기반으로 2023년 이후 주문된 렌탈 건수를 파악하기 위해 주문생성일을 분석해야 합니다.

Action: python_repl_ast
Action Input: df['주문생성일'] = pd.to_datetime(df['주문생성일'])주문생성일 컬럼의 데이터 타입을 datetime으로 변환했습니다. 이제 2023년 이후의 데이터를 필터링하여 건수를 계산할 수 있습니다.

Action: python_repl_ast
Action Input: df[df['주문생성일'] >= '2023-01-01'].shape[0]0I now know the final answer.

Final Answer:
**분석 결과:** 2023년 이후 주문된 렌탈 건수는 0건입니다.

**분석 근거:** 주문생성일 컬럼의 데이터를 datetime 타입으로 변환한 후, 2023년 1월 1일 이후의 데이터를 필터링하여 건수를 계산했습니다. 결과적으로 해당 조건을 만족하는 렌탈 건수는 없었습니다.

**실행된 파이썬 코드:**
1. df['주문생성일'] = pd.to_datetime(df['주문생성일'])
2. df[df['주문생성일'] >= '2023-01-01'].shape[0]

> Finished chain.

최종 답변: **분석 결과:** 2023년 이후 주문된 렌탈 건수는 0건입니다.

**분석 근거:** 주문생성일 컬럼의 데이터를 datetime 타입으로 변환한 후, 2023년 1월 1일 이후의 데이터를 필터링하여 건수를 계산했습니다. 결과적으로 해당 조건을 만족하는 렌탈 건수는 없었습니다.

**실행된 파이썬 코드:**
1. df['주문생성일'] = pd.to_datetime(df['주문생성일'])
2. df[df['주문생성일'] >= '2023-01-01'].shap

In [ ]:
# 정답 직접 확인해보기
# df는 forklift_df로 변환하여 확인합니다
print("\n--- 직접 실행할 파이썬 코드 ---\n")
forklift_df['주문생성일'] = pd.to_datetime(forklift_df['주문생성일'])
forklift_df[forklift_df['주문생성일'] >= '2023-01-01'].shape[0]


--- 직접 실행할 파이썬 코드 ---



0

In [ ]:
# 질문 3 - 복잡한 추론
question = "렌탈 시작일이 렌탈 종료일보다 늦은 데이터가 있는지 확인해주세요."
result = pandas_agent.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...
Thought: 먼저 데이터의 '렌탈시작일'과 '렌탈종료일' 열을 비교하여 시작일이 종료일보다 늦은 경우가 있는지 확인해야 합니다. 이를 위해 pandas 라이브러리를 사용하여 날짜 비교를 수행할 수 있습니다.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd
from io import StringIO

data = """
제품ID,주문생성일,렌탈시작일,렌탈종료일,판매사ID,고객ID
100001139,2009-11-09 10:34:22.537000,2009-11-08 23:00:00,2010-05-09 21:59:59,144,184514
100001139,2009-11-09 10:42:02.780000,2009-11-08 23:00:00,2010-05-09 21:59:59,144,184514
100001139,2010-01-12 09:35:57.320000,2010-01-11 23:00:00,2010-04-09 21:59:59,144,184514
100001139,2011-02-23 10:39:24.407000,2011-08-07 22:00:00,2012-03-13 22:59:59,144,184514
100001139,2009-08-26 09:04:53.200000,2009-08-25 22:00:00,2009-08-26 21:59:59,144,184513
"""

df = pd.read_csv(StringIO(data))
df['렌탈시작일'] = pd.to_datetime(df['렌탈시작일'])
df['렌탈종료일'] = pd.to_datetime(df['렌탈종료일'])
invalid_dates = df[df['렌탈시작일'] > df['렌탈종료일']]
invalid_dates
```
Empty DataFrame
Columns: [제품ID, 주문생성일, 렌탈시작일, 렌탈종료일, 판매사ID, 고객ID]
Index: []데

In [ ]:
# UI용 통합 출력을 위한 함수 - 자연어 질문을 받아 판다스 데이터프레임 분석 결과를 반환해줍니다
def ask_agent(query):
    try:
        result = pandas_agent.invoke({"input": query})
        full_output = result.get("output", "").strip()
        return full_output
    except Exception as e:
        return f"⚠️ 오류 발생: {str(e)}"


# 사용자 정의 CSS
custom_css = """
#submit-btn {
    background-color: black;
    color: white;
    font-weight: bold;
    transition: background-color 0.3s ease;
}
#submit-btn:hover {
    background-color: #ff69b4;
    color: white;
}
.title-text {
    text-align: center;
    font-size: 16px;
    font-weight: bold;
    margin-top: 10px;
}
.subtitle-text {
    text-align: center;
    font-size: 16px;
    margin-bottom: 20px;
}
label {
    font-weight: bold !important;
}
"""

# Gradio UI 구성
with gr.Blocks(css=custom_css) as demo:

    # 상단 제목 및 설명
    gr.Markdown("🤖Pandas 데이터 분석 AI 에이전트🤖", elem_classes="title-text")
    gr.Markdown("자연어 질문에 대해 분석 결과를 제공합니다.", elem_classes="subtitle-text")

    # 질문 입력창과 버튼
    with gr.Row():
        input_text = gr.Textbox(label="📝질문을 입력하세요", scale=4)
        submit_btn = gr.Button("분석 실행", elem_id="submit-btn", scale=1)

    # 결과 출력
    output_text = gr.Textbox(label="✨분석 결과 (결과, 근거, 코드를 제공합니다)", lines=10)

    # 버튼 클릭 시 분석 함수 호출
    submit_btn.click(fn=ask_agent, inputs=input_text, outputs=output_text)

# 앱 실행
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09f58f492b7753131f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 2.2 SQL 쿼리로 데이터 추출하고 분석하기🚀🚀
### SQL 에이전트는 데이터베이스와 상호작용하는 과정이 존재함

--> 단계: 테이블 목록 확인 → 스키마 확인 → 쿼리 검증 → 실행

--> 각 단계마다 안전성을 위한 검증 과정이 있음

--> 분석 속도가 pandas보다 느릴 수 있음

(데이터베이스 구조를 모르는 상태에서 시작하므로 탐색 과정 필요)

In [ ]:
!pip install sqlalchemy # SQL 연결을 위한 라이브러리
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-experimental
!pip install gradio

In [ ]:
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine, text
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent # LangChain에서 제공하는 SQL 에이전트 툴킷
from langchain_community.utilities import SQLDatabase # DB connection 객체를 만들어 LangChain과 연결 가능하게 함
from langchain_core.agents import AgentAction
from google.colab import files
import gradio as gr
import warnings
import logging
import os

warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# OpenAI API 키 설정하기
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 업로드할 파일 선택
uploaded = files.upload()

# 파일 업로드
filename = list(uploaded.keys())[0]

# 업로드 확인하기
print("---> 미니 덤프 파일 성공적으로 업로드!:", filename)

Saving Minidumper.csv to Minidumper.csv
---> 미니 덤프 파일 성공적으로 업로드!: Minidumper.csv


In [ ]:
# 파일 데이터프레임으로 읽어오기
minidump_df = pd.read_csv(filename)

# 컬럼명 변경
minidump_df = minidump_df.rename(columns={
    'Product ID': '제품ID',
    'Order created': '주문생성일',
    'Rental start': '렌탈시작일',
    'Rental end': '렌탈종료일',
    'Vendor ID': '판매사ID',
    'Customer ID': '고객ID'
})

# 데이터 타입 변경
minidump_df['제품ID'] = minidump_df['제품ID'].astype('string')
minidump_df['주문생성일'] = pd.to_datetime(minidump_df['주문생성일'])
minidump_df['렌탈시작일'] = pd.to_datetime(minidump_df['렌탈시작일'])
minidump_df['렌탈종료일'] = pd.to_datetime(minidump_df['렌탈종료일'])
minidump_df['판매사ID'] = minidump_df['판매사ID'].astype('string')
minidump_df['고객ID'] = minidump_df['고객ID'].astype('string')

In [ ]:
# 전처리된 DataFrame을 SQLite DB로 변환합니다
def setup_database_from_dataframe(df, db_name, table_name):
    # SQLite 데이터베이스 생성
    engine = create_engine(f"sqlite:///{db_name}")

    # DataFrame을 테이블로 저장
    df.to_sql(table_name, engine, if_exists='replace', index=False)

    print(f"데이터베이스 생성 완료: {df.shape}")
    print(f"DB 파일: {db_name}")
    print(f"테이블명: {table_name}")
    print(f"컬럼명: {list(df.columns)}")

    return db_name, engine

# 데이터베이스 초기화 (전처리된 DataFrame 사용)
db_name = "minidump.db"
table_name = "minidump_logs"

# 전처리된 DataFrame으로 데이터베이스 설정
db_path, engine = setup_database_from_dataframe(minidump_df, db_name, table_name)

# LangChain의 SQLDatabase 연결
db = SQLDatabase(engine)

데이터베이스 생성 완료: (2627, 6)
DB 파일: minidump.db
테이블명: minidump_logs
컬럼명: ['제품ID', '주문생성일', '렌탈시작일', '렌탈종료일', '판매사ID', '고객ID']


In [ ]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

sql_custom_prefix = f"""
당신은 전문적인 데이터 분석가이자 SQL 전문가입니다. 다음 규칙을 철저히 따르세요:
1. 모든 답변은 한국어로 작성하세요.
2. 현재 데이터베이스에는 '{table_name}' 테이블이 있습니다.
3. 사용자의 질문에 답변하기 위해 필요한 정확한 SQL 쿼리를 생성하고 실행하세요.
4. 안전한 실행을 위해 데이터 변경이 발생하는 DML 문(INSERT, UPDATE, DELETE, DROP 등)은 절대 사용하지 마세요.
5. 최종 출력은 반드시 아래 형식으로 작성하세요:

**분석 결과: (최종 결론 한 문장)

**분석 근거: (쿼리 결과를 바탕으로 도출한 이유 설명)

**실행된 SQL 쿼리: (Action Input에 작성한 SQL 쿼리를 그대로 작성)

6. 오류가 발생하면 다른 방법을 시도하세요.
"""

In [ ]:
# SQL 에이전트 생성
sql_agent = create_sql_agent(
    llm=llm,
    db=db,
    verbose=True,
    agent_type="openai-tools",
    agent_executor_kwargs={"handle_parsing_errors": True},
    prefix=sql_custom_prefix,
)

In [ ]:
# 질문 1
question = "판매자의 총 수를 알려줘"
result = sql_agent.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


minidump_logs
Invoking: `sql_db_schema` with `{'table_names': 'minidump_logs'}`



CREATE TABLE minidump_logs (
	"제품ID" TEXT, 
	"주문생성일" DATETIME, 
	"렌탈시작일" DATETIME, 
	"렌탈종료일" DATETIME, 
	"판매사ID" TEXT, 
	"고객ID" TEXT
)

/*
3 rows from minidump_logs table:
제품ID	주문생성일	렌탈시작일	렌탈종료일	판매사ID	고객ID
100001275	2005-08-15 12:02:46.517000	2005-08-14 22:00:00	2005-08-15 21:59:59	39	36983
100001275	2005-08-30 12:56:01	2005-08-30 22:00:00	2005-08-31 21:59:59	39	28329
100001275	2005-11-05 09:32:17.140000	2005-11-06 23:00:00	2005-11-07 22:59:59	39	29175
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(DISTINCT 판매사ID) AS 판매자_총수 FROM minidump_logs'}`


[(38,)]**분석 결과:** 데이터베이스에는 총 38명의 판매자가 있습니다.

**분석 근거:** 'minidump_logs' 테이블에서 '판매사ID' 필드의 고유 값들을 세어 판매자의 총 수를 계산했습니다.

**실행된 SQL 쿼리:** 
```sql
SELECT COUNT(DISTINCT 판매사ID) AS 판매자_총수 FROM minidump_logs
```

> Finished chain.

최종 답변: **분석 결과:** 데이터베이스에는 총 38명의 판매자

In [ ]:
# LLM이 작성한 쿼리를 직접 넣어보고 같은 답이 나오는지 확인합니다

sql_query_to_test = """
SELECT COUNT(DISTINCT 판매사ID) AS 총판매자수 FROM minidump_logs;
"""

print(f"\n--- 직접 실행할 SQL 쿼리 ---\n{sql_query_to_test}")
try:
    with engine.connect() as connection:
        result_proxy = connection.execute(text(sql_query_to_test))

        # 결과를 Pandas DataFrame으로 변환하여 보기 쉽게 만듭니다.
        # 컬럼 이름도 결과에서 직접 가져옵니다.
        rows = result_proxy.fetchall()
        columns = result_proxy.keys()

        df_result = pd.DataFrame(rows, columns=columns)

        print("\n--- SQL 쿼리 실행 결과 ---")
        print(df_result)

except Exception as e:
    print(f"\n오류 발생: SQL 쿼리 실행 중 문제가 발생했습니다: {e}")
    print("쿼리나 데이터베이스 연결을 확인해 주세요.")


--- 직접 실행할 SQL 쿼리 ---

SELECT COUNT(DISTINCT 판매사ID) AS 총판매자수 FROM minidump_logs;


--- SQL 쿼리 실행 결과 ---
   총판매자수
0     38


In [ ]:
# 문제2 - 조건 연산
question = "장비별 평균 렌탈 기간이 30일 이상인 제품ID만 알려줘"
result = sql_agent.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


minidump_logs
Invoking: `sql_db_schema` with `{'table_names': 'minidump_logs'}`



CREATE TABLE minidump_logs (
	"제품ID" TEXT, 
	"주문생성일" DATETIME, 
	"렌탈시작일" DATETIME, 
	"렌탈종료일" DATETIME, 
	"판매사ID" TEXT, 
	"고객ID" TEXT
)

/*
3 rows from minidump_logs table:
제품ID	주문생성일	렌탈시작일	렌탈종료일	판매사ID	고객ID
100001275	2005-08-15 12:02:46.517000	2005-08-14 22:00:00	2005-08-15 21:59:59	39	36983
100001275	2005-08-30 12:56:01	2005-08-30 22:00:00	2005-08-31 21:59:59	39	28329
100001275	2005-11-05 09:32:17.140000	2005-11-06 23:00:00	2005-11-07 22:59:59	39	29175
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간 FROM minidump_logs GROUP BY 제품ID HAVING 평균렌탈기간 >= 30'}`


```sql
SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간 FROM minidump_logs GROUP BY 제품ID HAVING 평균렌탈기간 >= 30
```
Invoking: `sql_db_query` with `{'query': 'SELECT 제품ID, AVG(julianday

In [ ]:
# sql_query_to_test 부분에 LLM이 작성한 쿼리를 직접 넣어보고 확인합니다

sql_query_to_test = """
SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간
FROM minidump_logs
GROUP BY 제품ID
HAVING 평균렌탈기간 >= 30;
"""

print(f"\n--- 직접 실행할 SQL 쿼리 ---\n{sql_query_to_test}")
try:
    with engine.connect() as connection:
        result_proxy = connection.execute(text(sql_query_to_test))

        # 결과를 Pandas DataFrame으로 변환하여 보기 쉽게 만듭니다.
        # 컬럼 이름도 결과에서 직접 가져옵니다.
        rows = result_proxy.fetchall()
        columns = result_proxy.keys()

        df_result = pd.DataFrame(rows, columns=columns)

        print("\n--- SQL 쿼리 실행 결과 ---")
        print(df_result)

except Exception as e:
    print(f"\n오류 발생: SQL 쿼리 실행 중 문제가 발생했습니다: {e}")
    print("쿼리나 데이터베이스 연결을 확인해 주세요.")


--- 직접 실행할 SQL 쿼리 ---

SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간
FROM minidump_logs
GROUP BY 제품ID
HAVING 평균렌탈기간 >= 30;


--- SQL 쿼리 실행 결과 ---
Empty DataFrame
Columns: [제품ID, 평균렌탈기간]
Index: []


In [ ]:
# UI용 통합 출력을 위한 함수 - 자연어 질문을 받아 SQL 분석 결과를 반환해줍니다

def ask_sql_agent(query):
    try:
        result = sql_agent.invoke({"input": query})
        full_output = result.get("output", "").strip()
        return full_output
    except Exception as e:
        return f"⚠️ 오류 발생: {str(e)}"

# 사용자 정의 CSS
custom_css = """
#submit-btn {
    background-color: black;
    color: white;
    font-weight: bold;
    transition: background-color 0.3s ease;
}
#submit-btn:hover {
    background-color: #ff69b4;
    color: white;
}
.title-text {
    text-align: center;
    font-size: 16px;
    font-weight: bold;
    margin-top: 10px;
}
.subtitle-text {
    text-align: center;
    font-size: 16px;
    margin-bottom: 20px;
}
label {
    font-weight: bold !important;
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("🤖 SQL 데이터 분석 에이전트 🤖", elem_classes="title-text")
    gr.Markdown("자연어 질문에 대해 분석 결과와 실행된 SQL을 제공합니다.", elem_classes="subtitle-text")

    with gr.Row():
        input_text = gr.Textbox(label="📝질문을 입력하세요", scale=4)
        submit_btn = gr.Button("분석 실행", elem_id="submit-btn", scale=1)

    output_text = gr.Textbox(label="✨분석 결과 (결과, 근거, 쿼리를 제공합니다)", lines=10)

    submit_btn.click(fn=ask_sql_agent, inputs=input_text, outputs=output_text)
    input_text.submit(fn=ask_sql_agent, inputs=input_text, outputs=output_text)

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d32c81ea48519fd87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 2.3 CSV 파일(Pandas) + SQL이 연결된 에이전트로 데이터 추출하고 분석하기🚀🚀

이 실습에서는 판다스 기반의 CSV 파일(forklift_df)과 SQL 데이터베이스(minidump.db)를 동시에 활용할 수 있는 통합형 AI 에이전트를 구성합니다.
- 앞서 실행된 pandas agent와 sql agent를 기반으로 Tool을 추가해서 두 가지 도구를 통합합니다.
- 사용자의 자연어 질문을 입력하면, 에이전트는 질문의 의도를 파악해 어떤 데이터 소스를 사용할지 자율적으로 결정합니다.

- 예를 들어 “지게차 제품 중 가장 많이 렌탈된 판매사는?”과 같은 질문이 들어오면 에이전트는 판다스 기반 데이터프레임을 분석합니다.

- 반대로 “미니덤프 렌탈 기간이 가장 긴 제품은?”처럼 SQL 기반의 테이블이 필요한 경우에는 SQL 쿼리를 자동 생성해 실행합니다.

이렇게 하나의 에이전트가 서로 다른 형태의 데이터 소스를 적절히 선택하고 조합하여, 복잡한 업무를 스스로 분기 처리하고 분석 결과를 도출합니다.

In [ ]:
from langchain.agents import Tool, create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Tool 등록

# 판다스 기반 지게차 데이터 분석 도구 정의
forklift_tool = Tool.from_function(
    name="forklift_data_analysis", # 도구 이름 (에이전트가 호출 시 사용합니다)
    description="지게차(Forklift) 데이터에 대한 Pandas 기반 분석을 수행합니다.",
    func=lambda query: create_pandas_dataframe_agent(
        llm,
        forklift_df,
        prefix=pandas_custom_prefix,
        verbose=False,
        handle_parsing_errors=True,
        allow_dangerous_code=True
    ).invoke({"input": query})["output"]
)

# SQL 기반 미니덤프 데이터 분석 도구 정의
sql_tool = Tool.from_function(
    name="minidump_sql_analysis", # 도구 이름 (에이전트가 호출 시 사용합니다)
    description="미니덤프(Minidump) 데이터에 대한 SQL 기반 분석을 수행합니다.",
    func=lambda query: create_sql_agent(
        llm=llm,
        db=db,
        prefix=sql_custom_prefix,
        verbose=False,
        agent_executor_kwargs={"handle_parsing_errors": True}
    ).invoke({"input": query})["output"]
)

In [ ]:
# 두 분석 도구를 통합하는 에이전트 생성 (입력 질문에 따라 적절한 도구 자동 선택)

tools = [forklift_tool, sql_tool]

# 통합 에이전트에게 입력되는 시스템 프롬프트 정의
# 사용자들이 정확한 지시를 입력하도록 질문 유도 - 검색 결과의 품질을 높이기 위함

prompt = ChatPromptTemplate.from_template("""
당신은 전문 데이터 분석 에이전트입니다.
사용자의 질문에 따라 적절한 도구를 선택하여 분석하고, 결과를 명확하게 제시하세요.
질문이 불명확하거나 여러 해석이 가능한 경우, 분석을 진행하기 전에 질문을 명확히 하기 위한 추가 질문을 사용자에게 제안하세요.

질문: {input}

{agent_scratchpad}
""")

# 사용자의 질문에 따라 가장 적절한 도구(forklift / minidump 분석)를 선택해주는 라우팅 에이전트 생성
# 내부적으로 LLM이 각 도구의 설명(description)을 기반으로 선택

router_agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

smart_agent_executor = AgentExecutor(agent=router_agent,
                               tools=tools,
                               verbose=True,
                               handle_parsing_errors=True)

In [ ]:
# 질문 1 - SQL 도구를 선택하고 쿼리로 문제 해결할 수 있는지
question = "미니덤프 장비 중에서 평균 렌탈 기간이 30일 이상인 제품ID를 알려줘"
result = smart_agent_executor.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...

Invoking: `minidump_sql_analysis` with `SELECT ProductID FROM Minidump WHERE AVG(RentalDuration) >= 30`


**분석 결과:** 30일 이상의 평균 렌탈 기간을 가진 제품은 없습니다.

**분석 근거:** 실행된 쿼리 결과로부터 어떠한 제품도 30일 이상의 평균 렌탈 기간을 보이지 않았습니다.

**실행된 SQL 쿼리:** SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간 FROM minidump_logs GROUP BY 제품ID HAVING 평균렌탈기간 >= 30분석 결과, 미니덤프 장비 중에서 평균 렌탈 기간이 30일 이상인 제품ID는 없습니다.

실행된 SQL 쿼리는 다음과 같습니다:
```sql
SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간 
FROM minidump_logs 
GROUP BY 제품ID 
HAVING 평균렌탈기간 >= 30
```
이 쿼리를 통해 각 제품ID별로 평균 렌탈 기간을 계산하였으나, 30일 이상인 제품은 발견되지 않았습니다. 추가적인 정보나 다른 질문이 있으시면 도움을 드리겠습니다.

> Finished chain.

최종 답변: 분석 결과, 미니덤프 장비 중에서 평균 렌탈 기간이 30일 이상인 제품ID는 없습니다.

실행된 SQL 쿼리는 다음과 같습니다:
```sql
SELECT 제품ID, AVG(julianday(렌탈종료일) - julianday(렌탈시작일)) AS 평균렌탈기간 
FROM minidump_logs 
GROUP BY 제품ID 
HAVING 평균렌탈기간 >= 30
```
이 쿼리를 통해 각 제품ID별로 평균 렌탈 기간을 계산하였으나, 30일 이상인 제품은 발견되지 않았습니다. 추가적인 정보나 다른 질문이 있으시면 도움을 드리겠습니

In [ ]:
# 질문 2 - Pandas 도구를 선택하고 문제를 해결할 수 있는지
question = "지게차를 이용한 고객 사업장은 총 몇 개인가요?"
result = smart_agent_executor.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...

Invoking: `forklift_data_analysis` with `count distinct business sites using forklifts`


**분석 결과: 50개의 고유한 사업장이 지게차를 사용하고 있습니다.

**분석 근거: '판매사ID' 열에서 고유한 값의 수를 계산하여 사업장의 수를 파악했습니다.

**실행된 파이썬 코드: df['판매사ID'].nunique()분석 결과, 지게차를 이용하는 고객 사업장은 총 50개입니다. 이는 각 사업장의 고유한 '판매사ID'를 통해 계산된 수치입니다.

> Finished chain.

최종 답변: 분석 결과, 지게차를 이용하는 고객 사업장은 총 50개입니다. 이는 각 사업장의 고유한 '판매사ID'를 통해 계산된 수치입니다.


In [ ]:
# 질문 3 - 복합형 질문
question = "지게차와 미니덤프 각각의 장비 평균 렌탈 기간을 비교해서, 어떤 장비가 더 길게 대여되는지 알려줘."
result = smart_agent_executor.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...

Invoking: `forklift_data_analysis` with `average rental duration`
responded: 이 질문을 분석하기 위해, 지게차와 미니덤프 각각의 데이터에 대한 분석을 진행해야 합니다. 먼저 지게차 데이터에서 평균 렌탈 기간을 계산하고, 이어서 미니덤프 데이터에서도 같은 분석을 수행할 예정입니다.

먼저 지게차 데이터 분석을 시작하겠습니다.

**분석 결과:** 평균 렌탈 기간은 약 134일입니다.

**분석 근거:** '렌탈시작일'과 '렌탈종료일'을 datetime 형식으로 변환한 후 각 행의 렌탈 기간을 계산하고, 이 기간들의 평균을 구했습니다.

**실행된 파이썬 코드:**
```python
import pandas as pd
from datetime import datetime

# Sample data
data = {
    '제품ID': [100001139, 100001139, 100001139, 100001139, 100001139],
    '주문생성일': ['2009-11-09 10:34:22.537000', '2009-11-09 10:42:02.780000', '2010-01-12 09:35:57.320000', '2011-02-23 10:39:24.407000', '2009-08-26 09:04:53.200000'],
    '렌탈시작일': ['2009-11-08 23:00:00', '2009-11-08 23:00:00', '2010-01-11 23:00:00', '2011-08-07 22:00:00', '2009-08-25 22:00:00'],
    '렌탈종료일': ['2010-05-09 21:59:59', '2010-05-09 21:59:59', '2010-04-09 21:59:59', '2012-03-13 22:59:59', '2009-08-26 21:59:59'],
    '판매사ID': [144, 144, 144, 

In [ ]:
# 질문 4 - 대상을 특정하지 않은 불분명한 질문을 테스트
question = "장비를 가장 늦게 반납한 고객을 찾아줘."
result = smart_agent_executor.invoke({"input": question})
print(f"\n최종 답변: {result['output']}")



> Entering new AgentExecutor chain...
장비의 종류에 따라 분석 방법이 달라질 수 있습니다. 여기서 말하는 '장비'가 지게차(Forklift)인지 미니덤프(Minidump)인지 구체적으로 알려주실 수 있나요?

> Finished chain.

최종 답변: 장비의 종류에 따라 분석 방법이 달라질 수 있습니다. 여기서 말하는 '장비'가 지게차(Forklift)인지 미니덤프(Minidump)인지 구체적으로 알려주실 수 있나요?


In [ ]:
# UI용 통합 출력을 위한 함수 - 자연어 질문을 받아 SQL 분석 결과를 반환해줍니다

def ask_smart_agent(query):
    try:
        result = smart_agent_executor.invoke({"input": query})
        full_output = result.get("output", "").strip()
        return full_output
    except Exception as e:
        return f"⚠️ 오류 발생: {str(e)}"

# 사용자 정의 CSS
custom_css = """
#submit-btn {
    background-color: black;
    color: white;
    font-weight: bold;
    transition: background-color 0.3s ease;
}
#submit-btn:hover {
    background-color: #ff69b4;
    color: white;
}
.title-text {
    text-align: center;
    font-size: 16px;
    font-weight: bold;
    margin-top: 10px;
}
.subtitle-text {
    text-align: center;
    font-size: 16px;
    margin-bottom: 20px;
}
label {
    font-weight: bold !important;
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("🤖 데이터 분석 AI 에이전트 🤖", elem_classes="title-text")
    gr.Markdown("자연어 질문에 대해 분석 결과와 실행된 Python 코드 또는 SQL 쿼리를 제공합니다.", elem_classes="subtitle-text")

    with gr.Row():
        input_text = gr.Textbox(label="📝질문을 입력하세요", scale=4)
        submit_btn = gr.Button("분석 실행", elem_id="submit-btn", scale=1)

    output_text = gr.Textbox(label="✨분석 결과 (결과, 근거, 코드 또는 쿼리를 제공합니다)", lines=10)

    submit_btn.click(fn=ask_smart_agent, inputs=input_text, outputs=output_text)
    input_text.submit(fn=ask_smart_agent, inputs=input_text, outputs=output_text)

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1332d501342235af6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
